<a href="https://colab.research.google.com/github/zlqhem/mlapi/blob/main/aws-ecr-torchscript/aws_ecr_torchscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# torchscript

## load a model from S3

In [ ]:
%%capture
!pip install boto3 python-dotenv

In [ ]:
# aws access key setup
import dotenv
# contains 'AWS_ACCESS_KEY', 'AWS_SCRET_ACCESS_KEY'
env_file = "/content/drive/MyDrive/w2/mlapi/aws.env"
dotenv.load_dotenv(env_file)

True

In [ ]:
try:
    import unzip_requirements
except ImportError:
    pass

import json
from io import BytesIO
import time
import os
import base64

import boto3
import numpy as np
from PIL import Image

import torch
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F

s3 = boto3.client("s3")
bucket = "soltware.test"
key = "v1/best.torchscript"

def download_model(s3, bucket, key):
    file_name = os.path.basename(key)
    print ('file_name', file_name)
    s3.download_file(bucket, key, file_name)


def load_model(s3, bucket):
  response = s3.get_object(Bucket=bucket, Key=key)
  #state = torch.load(BytesIO(response["Body"].read()))
  #model.load_state_dict(state)
  #model.eval()

  bytes_array = BytesIO(response["Body"].read())
  model = torch.jit.load(bytes_array, map_location=torch.device('cpu')).eval()
  return model

In [ ]:
download_model(s3, bucket, key)

file_name best.torchscript


In [ ]:
!ls -al best.torchscript

-rw-r--r-- 1 root root 103561564 Feb 13 02:24 best.torchscript


In [ ]:
!date

Sun Feb  4 04:14:51 PM UTC 2024


In [ ]:
model = load_model(s3, bucket)

In [ ]:
model

RecursiveScriptModule(original_name=DetectionModel)

## lambda handler

In [ ]:
classes = np.array([
  'Tomato Healthy',
  'Strawberry Healthy',
  'Lettuce Healthy',
  'Strawberry Ashy Mold',
  'Strawberry White Powdery Mildew',
  'Lettuce Bacterial Head Rot',
  'Lettuce Bacterial Wilt',
  'Tomato Leaf Mold',
  'Tomato Yellow Leaf Curl Virus',
])

def lambda_handler(event, context):
    '''
    if event.get("source") in ["aws.events", "serverless-plugin-warmup"]:
        print('Lambda is warm!')
        return {}
    '''

    data = json.loads(event["body"])
    print("data keys:", data.keys())
    image = data["image"]
    response = predict(input_fn_stream(image), model)
    return {
        'statusCode': 200,
        'body': json.dumps(response)
    }

def input_fn_stream(image):
    image = image[image.find(",")+1:]
    dec = base64.b64decode(image + "===")
    byte_array = BytesIO(dec)

    im = Image.open(byte_array).resize((640,640))
    im = im.convert("RGB")

    #https://dev.to/andreygermanov/how-to-create-yolov8-based-object-detection-web-service-using-python-julia-nodejs-javascript-go-and-rust-4o8e#prepare_the_input
    # "We do not need Alpha channel in the image for YOLOv8 predictions. Let's remove it"
    input = np.array(im)
    input = input.transpose(2,0,1)
    input = input.reshape(1,3,640,640)
    input = input/255.0
    return torch.Tensor(input)

def predict(img_tensor, model):
  predict_values = model(img_tensor)
  print(predict_values[0].shape)
  print('predict_values[0]', predict_values[0])
  preds = F.softmax(predict_values, dim=1)
  conf_score, indx = torch.max(preds, dim=1)
  conf_score = conf_score.cpu().numpy()
  indx = indx.cpu().numpy()
  predict_class = classes[indx]
  response = {}
  response['class'] = str(predict_class)
  response['confidence'] = str(conf_score)
  return response


## Use the deployed API


In [17]:
import base64
import json

path = "/content/drive/MyDrive/w2/flutter/strawberry-healthy.png"

with open(path, "rb") as image_file:
  encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

print (len(encoded_string))

#url = "https://zedisfh81b.execute-api.us-east-1.amazonaws.com/detect"
url = "https://qs9eqe954g.execute-api.us-east-1.amazonaws.com/detect"
data =  {
    "image": encoded_string
}

844988


In [ ]:
import requests
response = requests.post(url, json=data)
response

<Response [200]>

In [10]:
print(response.headers)

{'Date': 'Wed, 14 Feb 2024 23:18:10 GMT', 'Content-Type': 'text/plain; charset=utf-8', 'Content-Length': '227', 'Connection': 'keep-alive', 'Apigw-Requestid': 'TJiE1hRYoAMEJ_A='}


In [13]:
response.json()

{'class': "[['Lettuce Healthy' 'Lettuce Healthy' 'Lettuce Healthy' ...\n  'Strawberry Healthy' 'Strawberry Healthy' 'Tomato Healthy']]",
 'confidence': '[[0.9999974  1.         1.         ... 1.         1.         0.99998224]]'}

In [16]:

d = response.json()
print(d.keys())
print(d['class'])
print(d['confidence'])

dict_keys(['class', 'confidence'])
[['Lettuce Healthy' 'Lettuce Healthy' 'Lettuce Healthy' ...
  'Strawberry Healthy' 'Strawberry Healthy' 'Tomato Healthy']]
[[0.9999974  1.         1.         ... 1.         1.         0.99998224]]


## references

* https://aws.amazon.com/ko/blogs/machine-learning/using-container-images-to-run-pytorch-models-in-aws-lambda/
* https://github.com/ahmedbesbes/cartoonify/tree/main